In [2]:
#py Script for CHE5037 Fluidized Drying Bed experiment
#Kyle Brittingham & John Lehman - Fall 2020
#Quickly calculates some psychrometric values from an excel sheet then outputs the data and saves the sheet

#use Python 3.X and install 2 libraries--openpyxl and psychrolib

#save the data as .xlsx and verify the data is in the right columns:
#inputs in columns: D=T1 dry bulb temp, E=T2 wet bulb temp,F=T3 hot dry entry, G=T4 exit temp, H=%RH exit
#output in columns: M=humidity ratio inlet, N=specific vol inlet, O=humidity ratio outlet, P=specific vol outlet

#setup libraries
#https://openpyxl.readthedocs.io/en/stable/
import openpyxl 
from pathlib import Path

#https://github.com/psychrometrics/psychrolib/blob/master/docs/overview.md
import psychrolib 
psychrolib.SetUnitSystem(psychrolib.SI) #celcius, pascals, etc...

#setup to import the file
File_name = '20201103.xlsx' #this is the name of the data sheet WITH .xlsx at the end
Path_name = 'D:\\Documents\\py\\megan\\'#the first part is the path to the file!
xlsx_file = Path(Path_name, File_name)
print(xlsx_file) #target path and file

#import file into py
wb_obj = openpyxl.load_workbook(xlsx_file)
print(wb_obj)

#open and save your data on the sheet you want to calculate on
sheet = wb_obj.active 
print(sheet) #this is the sheet name
print("Number of Rows: " + str(sheet.max_row)) #this is will set the for loop length

P = 1 #atm
P_pascals = 101325*P

Row_start = 2 #this is the EXCEL ROW # the loop will start on--YOU MAY NEED TO CHANGE THIS VALUE
MaxRow = sheet.max_row #ending point for the loop

#inputs in columns: D=T1 dry bulb temp, E=T2 wet bulb temp,F=T3 hot dry entry, G=T4 exit temp, H=%RH exit
#output in columns: M=humidity ratio inlet, N=specific vol inlet, O=humidity ratio outlet, P=specific vol outlet

#for loop to calc values; calc row then loop to next row
for i in range(Row_start, MaxRow+1):   #arrays start at 0 in python, but start at 1 in openpyxl functions
    
    T1 = sheet.cell(row = i, column = 4).value
    T2 = sheet.cell(row = i, column = 5).value
    if T1 < T2:
        print(i, T1, T2) #this tells you if the wet bulb temp > dry which is a no-no---highly recommend to just use an avg wet bulb temp
    
    #M from D&E
    hum_ratio_before = psychrolib.GetHumRatioFromTWetBulb(T1, T2, P_pascals) #dry, wet, pressure
    #N from D&E
    specific_vol_before = psychrolib.GetMoistAirVolume(T1, hum_ratio_before, P_pascals) #dry, hum-ratio, pressure

    T4 = sheet.cell(row = i, column = 7).value
    RH = sheet.cell(row = i, column = 8).value /100
    #O from G&H
    hum_ratio_after = psychrolib.GetHumRatioFromRelHum(T4, RH, P_pascals) #dry, RH, pressure
    #P from G&H
    specific_vol_after = psychrolib.GetMoistAirVolume(T4, hum_ratio_after, P_pascals) #dry, hum-ratio, pressure

    #output the calc into the sheet--careful of units
    sheet.cell(i, 13).value = hum_ratio_before    #kg/kg
    sheet.cell(i, 14).value = specific_vol_before #m3/kg
    sheet.cell(i, 15).value = hum_ratio_after     #kg/kg
    sheet.cell(i, 16).value = specific_vol_after  #m3/kg

#save the file!
wb_obj.save(Path_name + File_name)
print(Path_name + File_name)
print('Complete!') #hooray!

D:\Documents\py\megan\20201103.xlsx
<Worksheet "Edited">
Number of Rows: 1533
D:\Documents\py\megan\20201103.xlsx
Complete!
